In [32]:
import pandas as pd
import networkx as nx

# Leer los datos de colaboraciones de la hoja 'author_df'
author_df = pd.read_excel('./all_data_ai_data_engineering_1.xlsx', sheet_name='author_df')

# Crear una columna que represente el par de autores, ordenados alfabéticamente para evitar duplicados
author_df['pair'] = author_df.apply(lambda row: tuple(sorted([row['from'], row['to']])), axis=1)

# Contar las colaboraciones para cada par de autores
collaboration_counts = author_df['pair'].value_counts().to_dict()

# Crear el grafo de NetworkX basado en los datos de colaboraciones
G = nx.Graph()

# Agregar cada autor y la fuerza de conexión a la red
for (author1, author2), count in collaboration_counts.items():
    G.add_edge(author1, author2, weight=count)

# (Numero de Autores, Numero de Conexiones)
num_nodes = G.number_of_nodes()
num_edges = G.number_of_edges()
num_nodes, num_edges

(41444, 141592)

In [36]:
# Nombres de Autor
authorInDeph = "HE K"

# Contador de Conexiones Unicas de Autor
G.degree(authorInDeph)

# Visualizador de Veciones (Con Peso)
# En este caso, el peso es el número de conexiones encontradas para cada par de autores
collaborators_with_weights = [(neighbor, G[authorInDeph][neighbor].get('weight', 1)) for neighbor in G.neighbors(authorInDeph)]
print(collaborators_with_weights)


[('REN S', 28), ('SUN J', 27), ('ZHANG X', 24), ('GIRSHICK R', 8), ('BELONGIE S', 3), ('HARIHARAN B', 3), ('DOLLÁR P', 3), ('WANG J', 2), ('DOLLAR P', 1), ('LIN T-Y', 1), ('TU Z', 1), ('XIE S', 1), ('LIN T', 1), ('TIAN Y', 1), ('SUN C', 1), ('TIAN G', 1), ('ZHANG Q', 1), ('ZHENG Y', 1), ('ZHENG B', 1), ('XUE T', 1), ('ZHAO H', 1), ('TONG D', 1), ('GENG G', 1), ('DAVIS S', 1), ('LI M', 1), ('LIU F', 1), ('HONG C', 1), ('YUAN Q', 1), ('LIU W', 1), ('CAI J', 1), ('HU W', 1), ('TIANBO S', 1), ('ZOU Y', 1), ('PAN J', 1), ('JI L', 1), ('YANG Q', 1), ('ASGHAR MR', 1), ('KIM DD', 1), ('LIN TY', 1), ('WANG G', 1), ('ARUMUGASAMY SK', 1), ('JEWARATNAM J', 1), ('CHUNG CH', 1), ('SHIMIZU Y', 1), ('WONG YJ', 1)]


In [37]:
import pandas as pd
import networkx as nx

def calcular_metricas_investigadores(filepath: str) -> str:
    """
    Calcula métricas de red para cada investigador en un archivo de datos de colaboraciones.
    Las métricas calculadas son: huecos estructurales, fortaleza promedio de enlaces, y número de conexiones.

    Parameters:
    filepath (str): Ruta al archivo Excel que contiene la hoja de datos de colaboraciones.

    Returns:
    str: Ruta al archivo Excel generado con las métricas calculadas.
    """

    # Leer los datos de colaboraciones de la hoja 'author_df'
    author_df = pd.read_excel(filepath, sheet_name='author_df')

    # Crear una columna que represente el par de autores, ordenados alfabéticamente para evitar duplicados
    author_df['pair'] = author_df.apply(lambda row: tuple(sorted([row['from'], row['to']])), axis=1)

    # Contar el número de colaboraciones para cada par de autores
    collaboration_counts = author_df['pair'].value_counts().to_dict()

    # Crear el grafo de NetworkX basado en los datos de colaboraciones
    G = nx.Graph()
    for (author1, author2), count in collaboration_counts.items():
        G.add_edge(author1, author2, weight=count)

    # Inicializar una lista para almacenar los datos de cada investigador
    metrics_data = []

    # Iterar sobre cada investigador en la red para calcular las métricas
    for author in G.nodes:
        # Obtener el número de conexiones directas del autor (grado en el grafo)
        num_connections = G.degree(author)

        # Calcular los huecos estructurales de manera individual para cada nodo
        structural_holes = nx.constraint(G, nodes=[author]).get(author, None)

        # Calcular la fortaleza del enlace
        max_collaborations = max(
            (G[author][neighbor].get('weight', 1) for neighbor in G.neighbors(author)),
            default=1
        )

        total_strength = 0
        count = 0

        for neighbor in G.neighbors(author):
            collaborations = G[author][neighbor].get('weight', 1)
            normalized_strength = collaborations / max_collaborations
            total_strength += normalized_strength
            count += 1

        # Calcular la fortaleza promedio del enlace
        link_strength = total_strength / count if count > 0 else 0

        # Imprimir resultados parciales
        print(f"Nombre del Investigador: {author}, Huecos estructurales: {structural_holes}, Fortaleza del enlace: {link_strength}, Número de conexiones: {num_connections}")

        # Almacenar los resultados de las métricas en la lista
        metrics_data.append({
            'Nombre del Investigador': author,
            'Huecos estructurales': structural_holes,
            'Fortaleza del enlace': link_strength,
            'Número de conexiones': num_connections
        })

    # Crear un DataFrame con los resultados
    metrics_df = pd.DataFrame(metrics_data)

    # Guardar los resultados en un archivo Excel
    output_path = './investigadores_metricas.xlsx'
    metrics_df.to_excel(output_path, index=False)

    return output_path

# Ejecutar la función con la ruta al archivo Excel
calcular_metricas_investigadores('all_data_ai_data_engineering_1.xlsx')


Nombre del Investigador: HE K, Huecos estructurales: 0.04055489965641541, Fortaleza del enlace: 0.10714285714285705, Número de conexiones: 45
Nombre del Investigador: REN S, Huecos estructurales: 0.0994062445046565, Fortaleza del enlace: 0.2372448979591836, Número de conexiones: 14
Nombre del Investigador: BENGIO Y, Huecos estructurales: 0.08442562860036519, Fortaleza del enlace: 0.10255102040816316, Número de conexiones: 71
Nombre del Investigador: LECUN Y, Huecos estructurales: 0.06176361451140616, Fortaleza del enlace: 0.13157894736842096, Número de conexiones: 38
Nombre del Investigador: SUN J, Huecos estructurales: 0.018511426346920427, Fortaleza del enlace: 0.07407407407407422, Número de conexiones: 100
Nombre del Investigador: CHAI C, Huecos estructurales: 0.03751854078371395, Fortaleza del enlace: 0.11487179487179493, Número de conexiones: 39


KeyboardInterrupt: 

In [31]:
filepath = "./all_data_ai_data_engineering_1.xlsx"

# Leer los datos de colaboraciones de la hoja 'author_df'
author_df = pd.read_excel(filepath, sheet_name='author_df')

# Crear una columna que represente el par de autores, ordenados alfabéticamente para evitar duplicados
author_df['pair'] = author_df.apply(lambda row: tuple(sorted([row['from'], row['to']])), axis=1)

# Contar el número de colaboraciones para cada par de autores
collaboration_counts = author_df['pair'].value_counts().to_dict()

collaboration_counts

{('HE K', 'REN S'): 28,
 ('BENGIO Y', 'LECUN Y'): 28,
 ('REN S', 'SUN J'): 27,
 ('HE K', 'SUN J'): 27,
 ('CHAI C', 'LI G'): 25,
 ('SUN J', 'ZHANG X'): 25,
 ('REN S', 'ZHANG X'): 24,
 ('HE K', 'ZHANG X'): 24,
 ('KRIZHEVSKY A', 'SUTSKEVER I'): 24,
 ('HASTIE T', 'TIBSHIRANI R'): 19,
 ('HINTON GE', 'KRIZHEVSKY A'): 19,
 ('HINTON GE', 'SUTSKEVER I'): 18,
 ('CHU X', 'ILYAS IF'): 17,
 ('STOICA I', 'ZAHARIA M'): 17,
 ('HINTON G', 'LECUN Y'): 16,
 ('BENGIO Y', 'HINTON G'): 16,
 ('LOPEZ D', 'MANOGARAN G'): 15,
 ('BENGIO Y', 'CHO K'): 15,
 ('IBRAHIM A', 'VALLE M'): 15,
 ('SIMONYAN K', 'ZISSERMAN A'): 14,
 ('BENGIO Y', 'COURVILLE A'): 14,
 ('PEDREGOSA F', 'VAROQUAUX G'): 14,
 ('ABADI M', 'BARHAM P'): 14,
 ('ABADI M', 'CHEN Z'): 14,
 ('SHENKER S', 'STOICA I'): 14,
 ('BARHAM P', 'CHEN Z'): 14,
 ('LI G', 'TANG N'): 13,
 ('GRAMFORT A', 'VAROQUAUX G'): 13,
 ('ELOUATAOUI W', 'GAHI Y'): 13,
 ('FRIEDMAN J', 'HASTIE T'): 13,
 ('FRIEDMAN J', 'TIBSHIRANI R'): 13,
 ('GRAMFORT A', 'PEDREGOSA F'): 13,
 ('BIANCH